In [1]:
include("../src/main.jl");
include("helper_functions_rigidity.jl");

In [2]:
# Laurent polynomial system
QQa, (lambda12,lambda13,lambda23,lambda24,lambda34) = polynomial_ring(QQ,["λ12","λ13","λ23","λ24","λ34"])
QQaxpm, (x12,x13,x23,x24,x34) = laurent_polynomial_ring(QQa,["x12","x13","x23","x24","x34"])

F_laurent = [x12+x23-x13,
    x23+x34-x24, 
    lambda12*x12^-1+lambda23*x23^-1-lambda13*x13^-1,
    lambda23*x23^-1+lambda34*x34^-1-lambda24*x24^-1, 
    x23-1
]

display(F_laurent)

# Conver to polynomial system
F = [f.mpoly for f in F_laurent]
QQax = parent(first(F))

5-element Vector{AbstractAlgebra.Generic.LaurentMPolyWrap{QQMPolyRingElem, AbstractAlgebra.Generic.MPoly{QQMPolyRingElem}, AbstractAlgebra.Generic.LaurentMPolyWrapRing{QQMPolyRingElem, AbstractAlgebra.Generic.MPolyRing{QQMPolyRingElem}}}}:
 x12 - x13 + x23
 x23 - x24 + x34
 λ23*x23^-1 - λ13*x13^-1 + λ12*x12^-1
 λ34*x34^-1 - λ24*x24^-1 + λ23*x23^-1
 x23 - 1

Multivariate polynomial ring in 5 variables x12, x13, x23, x24, x34
  over multivariate polynomial ring in 5 variables over QQ

In [3]:
# Construct the graph
G = Graph{Undirected}(4)
add_edge!(G,1,2) # 1
add_edge!(G,1,3) # 2
add_edge!(G,2,3) # 3
add_edge!(G,2,4) # 4
add_edge!(G,3,4) # 5

# Construct the edge matrix
edgeMatrix = zeros(QQ,nv(G),ne(G))
for (i,edge) in enumerate(edges(G))
    edgeMatrix[dst(edge),i] = 1
    edgeMatrix[src(edge),i] = -1
end

In [4]:
# Pertubation of system
v = [3,4,5,6,7]
QQt, t  = rational_function_field(QQ,"t")
QQtx, (x12,x13,x23,x24,x34) = polynomial_ring(QQt,symbols(QQax))
phi = hom(QQax, QQtx, c -> evaluate(c, t.^v), gens(QQtx))
F_specialized = phi.(F)

5-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQPolyRingElem}}}:
 x12 - x13 + x23
 x23 - x24 + x34
 t^5*x12*x13 - t^4*x12*x23 + t^3*x13*x23
 t^7*x23*x24 - t^6*x23*x34 + t^5*x24*x34
 x23 - 1

In [5]:
# Tropicalizations
Sigma = tropical_linear_space(edgeMatrix)
SigmaInverted = -Sigma
SigmaInvertedShifted = SigmaInverted + v

Min tropical variety

In [6]:
# Check transersality and compute intersection
# This should be inluded in Oscar in the future
nonEmptyIntersections = []
intersectionMultiplicities = ZZRingElem[]
for (i,sigma) in enumerate(maximal_polyhedra(Sigma))
    for (j,tau) in enumerate(maximal_polyhedra(SigmaInvertedShifted))
        sigmaTau = intersect(sigma,tau)
        @assert (dim(sigmaTau)<0)||(dim(sigmaTau)==1) "shift non-generic"
        if (dim(sigmaTau)==1)
            push!(nonEmptyIntersections,sigmaTau)
            push!(intersectionMultiplicities,Oscar.tropical_intersection_multiplicity(sigma,tau))
        end
    end
end

pts = Vector{QQFieldElem}[]
for sigma in nonEmptyIntersections
    w = first(minimal_faces(sigma).base_points)
    w = w .- w[3]
    push!(pts,Vector(w))
end

display(pts)

4-element Vector{Vector{QQFieldElem}}:
 [0, 1, 0, 1, 0]
 [0, 1, 0, 0, 2]
 [-2, -2, 0, 1, 0]
 [-2, -2, 0, 0, 2]

In [7]:
# Computation of homotopies
for w in pts 
    display(w)
    display(homotopy_from_tropical_data(F_specialized, w))
    println()
end

5-element Vector{QQFieldElem}:
 0
 1
 0
 1
 0

5-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQPolyRingElem}}}:
 x12 - t*x13 + x23
 x23 - t*x24 + x34
 t^2*x12*x13 - x12*x23 + x13*x23
 t^2*x23*x24 - x23*x34 + x24*x34
 x23 - 1

5-element Vector{QQFieldElem}:
 0
 1
 0
 0
 2

5-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQPolyRingElem}}}:
 x12 - t*x13 + x23
 x23 - x24 + t^2*x34
 t^2*x12*x13 - x12*x23 + x13*x23
 x23*x24 - t*x23*x34 + x24*x34
 x23 - 1

5-element Vector{QQFieldElem}:
 -2
 -2
 0
 1
 0

5-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQPolyRingElem}}}:
 x12 - x13 + t^2*x23
 x23 - t*x24 + x34
 x12*x13 - t*x12*x23 + x13*x23
 t^2*x23*x24 - x23*x34 + x24*x34
 x23 - 1

5-element Vector{QQFieldElem}:
 -2
 -2
 0
 0
 2

5-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQPolyRingElem}}}:
 x12 - x13 + t^2*x23
 x23 - x24 + t^2*x34
 x12*x13 - t*x12*x23 + x13*x23
 x23*x24 - t*x23*x34 + x24*x34
 x23 - 1